In [1]:
from xbbg import blp
import pdblp
import workdays
import datetime
from sklearn.cluster import KMeans
import pandas as pd

LS = ["ES","NQ","RTY","VG","GX","Z ","PT","XP","TP","HI","XU","IH"]

T=[]
T1=[]
for i in range(len(LS)):
    T += [LS[i]+"A Index"]
    T1 += [LS[i]+"1 Index"]
    
BDP = blp.bdp(tickers=T, flds=["name","currency","undl_spot_ticker"])
BDP = BDP.loc[T,:]

BDP1 = blp.bdp(tickers=T1, flds=["volume_avg_5d","volatility_90d"])
BDP1 = BDP1.loc[T1,:]

undl = BDP.loc[T,["undl_spot_ticker"]].loc[T,:] + " Index"

tmp=[]
for i in range(len(undl)):
    tmp += [undl.iloc[i,0]]
tmp = blp.bdp(tickers=tmp, flds="country_iso").loc[tmp,"country_iso"]
BDP["country"] = BDP["name"]
for i in range(len(T)):
    BDP["country"][i] = tmp[i]
    
    undl = BDP.loc[T,["undl_spot_ticker"]].loc[T,:] + " Index"
tmp=[]
for i in range(len(undl)):
    tmp += [undl.iloc[i,0]]
tmp = blp.bdp(tickers=tmp, flds="country_iso").loc[tmp,"country_iso"]

for i in range(len(T)):
    BDP["country"][i] = tmp[i]
BDP["volume"] = BDP1["volume_avg_5d"].values
BDP["vola"] = BDP1["volatility_90d"].values
BDP["vola"] = round(BDP["vola"],1)
BDP.insert(0, "Ticker", LS)
BDP.index = list(range(len(BDP)))
BDP

,Ticker,name,currency,undl_spot_ticker,country,volume,vola
0,ES,S&P500 EMINI FUT Sep23,USD,SPX,US,1883680.6,14.3
1,NQ,NASDAQ 100 E-MINI Sep23,USD,NDX,US,561028.6,18.5
2,RTY,E-Mini Russ 2000 Sep23,USD,RTY,US,274011.0,21.7
3,VG,EURO STOXX 50 Jun23,EUR,SX5E,EU,1449084.6,16.2
4,GX,DAX INDEX FUTURE Jun23,EUR,DAX,DE,68603.4,15.3
5,Z,FTSE 100 IDX FUT Jun23,GBP,UKX,GB,255237.4,13.4
6,PT,S&P/TSX 60 IX FUT Jun23,CAD,SPTSX60,CA,77549.6,11.4
7,XP,SPI 200 FUTURES Sep23,AUD,AS51,AU,97747.4,11.3
8,TP,TOPIX INDX FUTR Sep23,JPY,TPX,JP,79638.6,14.7
9,HI,HANG SENG IDX FUT Jun23,HKD,HSI,HK,105121.2,22.9


In [2]:
#Get Historidcal Data
con = pdblp.BCon(timeout=5000)
con.start()

d_from = workdays.workday(datetime.datetime.today(), days=-260).strftime("%Y%m%d")
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

df = con.bdh(T1, ["px_last"], d_from, d_to).reset_index()
df = df[["date"]+T1]
df.columns = ["Date"]+LS

for j in range(len(LS)):    
    df[LS[j]+"_RetD"]=df[LS[j]].pct_change()
df

,Date,ES,NQ,RTY,VG,GX,Z,PT,XP,TP,...,RTY_RetD,VG_RetD,GX_RetD,Z _RetD,PT_RetD,XP_RetD,TP_RetD,HI_RetD,XU_RetD,IH_RetD
0,2022-06-16,3789.49,11600.60,1701.54,3365.8,13201.15,7001.0,1149.57,6405.9,1814.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-17,3794.13,11745.65,1715.13,3366.7,13331.10,6950.5,1146.66,6310.7,1773.5,...,0.007987,0.000267,0.009844,-0.007213,-0.002531,-0.014861,-0.022542,0.012077,0.019415,0.000392
2,2022-06-20,NaN,NaN,NaN,3393.3,13468.52,7064.0,1162.51,6287.9,1771.6,...,0.000000,0.007901,0.010308,0.016330,0.013823,-0.003613,-0.001071,0.005658,0.006538,0.003036
3,2022-06-21,3889.10,12037.29,1744.28,3420.8,13506.19,7095.5,1166.02,6361.3,1804.2,...,0.016996,0.008104,0.002797,0.004459,0.003019,0.011673,0.018401,0.017066,0.000282,0.016957
4,2022-06-22,3883.94,12025.33,1742.22,3397.2,13373.85,7046.0,1153.58,6335.5,1801.3,...,-0.001181,-0.006899,-0.009798,-0.006976,-0.010669,-0.004056,-0.001607,-0.023008,-0.009776,-0.014435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2023-06-08,4342.18,14681.84,1901.24,4303.0,16007.00,7606.5,1197.60,7064.0,2190.0,...,-0.004810,0.001396,0.001752,-0.003080,-0.001251,-0.003639,-0.004545,0.001717,0.012634,-0.004864
256,2023-06-09,4348.75,14735.75,1886.20,4291.0,15957.00,7565.0,1195.60,7083.9,2219.5,...,-0.007911,-0.002789,-0.003124,-0.005456,-0.001670,0.002817,0.013470,0.005766,0.000480,-0.005903
257,2023-06-12,4388.00,14986.75,1895.10,4317.0,16091.00,7568.0,1197.20,7058.0,2236.5,...,0.004718,0.006059,0.008398,0.000397,0.001338,-0.003656,0.007659,0.002118,0.007034,0.004756
258,2023-06-13,4416.75,15099.00,1916.70,4354.0,16235.00,7601.5,1201.90,7107.0,2263.0,...,0.011398,0.008571,0.008949,0.004427,0.003926,0.006942,0.011849,0.005824,0.004525,0.004626


In [3]:
#K-means
tmp = [x + "_RetD" for x in LS]
df_Ret = df.loc[1:,tmp].reset_index(drop=True)

c = KMeans(n_clusters=4,random_state=3).fit_predict(df_Ret.T) #pred_cluster
# c = KMeans(n_clusters=4,init='k-means++',random_state=3).fit_predict(df_Ret.T)

df2 = pd.DataFrame(BDP.values)
df2.columns = BDP.columns

df2.insert(0,"Group",c)
df2 = df2.sort_values(by='Group').reset_index(drop=True)
df2

,Group,Ticker,name,currency,undl_spot_ticker,country,volume,vola
0,0,XP,SPI 200 FUTURES Sep23,AUD,AS51,AU,97747.4,11.3
1,0,TP,TOPIX INDX FUTR Sep23,JPY,TPX,JP,79638.6,14.7
2,0,IH,SGX Nifty 50 Jun23,USD,NIFTY,IN,50874.8,9.7
3,1,VG,EURO STOXX 50 Jun23,EUR,SX5E,EU,1.44908e+06,16.2
4,1,GX,DAX INDEX FUTURE Jun23,EUR,DAX,DE,68603.4,15.3
5,1,Z,FTSE 100 IDX FUT Jun23,GBP,UKX,GB,255237,13.4
6,1,PT,S&P/TSX 60 IX FUT Jun23,CAD,SPTSX60,CA,77549.6,11.4
7,2,ES,S&P500 EMINI FUT Sep23,USD,SPX,US,1.88368e+06,14.3
8,2,NQ,NASDAQ 100 E-MINI Sep23,USD,NDX,US,561029,18.5
9,2,RTY,E-Mini Russ 2000 Sep23,USD,RTY,US,274011,21.7
